Устанавливаем и импортируем необходимые библиотеки

In [ ]:
#Установка
!pip install folium
!pip install geopy
!pip install gmaps

!pip install osmnx

Для корректной работы необходимо перезапустить среду выполнения после установки пакетов и библиотек.



---



In [41]:
#Библиотеки
import numpy as np
import pandas as pd

import folium
from folium import plugins
import gmaps

from geopy.geocoders import Nominatim

import requests

import re
import json

import osmnx as ox
import networkx as nx

import warnings

**Функция определяет центральную координату.**



---



Функция является основной по поиску координаты центральной точки (Барицентр).

Суть функции в подсчёте средней координаты по X и Y.

In [42]:
#Центр по координатам
def center(adresses):

  """
  Barycenter
  The essence of the function is to calculate the average X and Y coordinates.
  Output -> tuple(latitude, longitude)
  """

  list_x = []
  list_y = []

  for i in range(len(adresses)):
    list_x.append(float(adresses[i][0]))
    list_y.append(float(adresses[i][1]))

  x = np.array(list_x)
  y = np.array(list_y)
  center_point = [np.mean(x, axis=0), np.mean(y, axis=0)]

  return center_point

**Адресная книга**



---



Создаем датасет с пользователями и адресами.

Пользователи вводят данные по своей геолокации и формируется датасет данных.

In [43]:
#Создаем пустой датасет

adress_book = pd.DataFrame()
adress_book['User_Name'] = None
adress_book['City'] = None
adress_book['Street'] = None
adress_book['House_Number'] = None
adress_book['Building'] = None
adress_book['Full_adress'] = None

adress_book['Coordinates'] = None

In [44]:
n = 0

while True:

  """
  Users enter data on their geolocation and a dataset is formed.
  """

  n += 1
  flag = input('Добавить участника? (Do you want to add a user?) ')
  if flag == 'да' or flag == 'Да' or flag == 'Yes' or flag == 'yes':

    adress_book.loc[n, 'User_Name'] = input('Введите Ваше имя (Name):')
    adress_book.loc[n, 'City'] = input('Введите город (City):')
    adress_book.loc[n, 'Street'] = input('Введите улицу (Street):')
    adress_book.loc[n, 'House_Number'] = input('Введите номер дома (House number):')
    adress_book.loc[n, 'Building'] = input('Введите корпус (если есть) (Building):')


    if adress_book.loc[n, 'Building'] == '':
      adress_book.loc[n, 'Full_adress'] = adress_book.loc[n, 'City'] + ' ' + adress_book.loc[n, 'Street'] + ' ' + adress_book.loc[n, 'House_Number']
    else:
      adress_book.loc[n, 'Full_adress'] = adress_book.loc[n, 'City'] + ' ' + adress_book.loc[n, 'Street'] + ' ' + adress_book.loc[n, 'House_Number'] + ' ' + adress_book.loc[n, 'Building']
    print('\n')

  else:
    break

Добавить участника? (Do you want to add a user?) да
Введите Ваше имя (Name):Василий
Введите город (City):Санкт-Петербург
Введите улицу (Street):Есенина
Введите номер дома (House number):8
Введите корпус (если есть) (Building):1


Добавить участника? (Do you want to add a user?) да
Введите Ваше имя (Name):Иван
Введите город (City):Санкт-Петербург
Введите улицу (Street):Федора Абрамова
Введите номер дома (House number):1
Введите корпус (если есть) (Building):


Добавить участника? (Do you want to add a user?) да
Введите Ваше имя (Name):Пётр
Введите город (City):Санкт-Петербург
Введите улицу (Street):Ленина
Введите номер дома (House number):5
Введите корпус (если есть) (Building):


Добавить участника? (Do you want to add a user?) нет


**Получаем координаты по адресам**



---



Делаем запрос в GoogleMaps по адресу и обрабатываем ответ. 

Вытягиваем координаты в теле ответа.

Добавляем координаты в датасет.

In [45]:
for i in range(len(adress_book)):

  #Делаем запрос по адресу
  r = requests.get('https://www.google.com/maps/search/' + adress_book.loc[i+1, 'Full_adress'])

  #Регулярными выражениями находим координаты в тексте запроса
  coord = re.findall(r'center=.+?&', r.text)[0][7:-1]

  #Добавляем координаты в датасет
  latitude = re.findall(r'.+%', coord)[0][:-1]
  longitude = re.findall(r'C.+', coord)[0][1:]

  adress_book.loc[i+1, 'Coordinates'] = (latitude, longitude)

In [46]:
#Итоговый датасет с координатами
#Final dataset with coordinates

adress_book

,User_Name,City,Street,House_Number,Building,Full_adress,Coordinates
1,Василий,Санкт-Петербург,Есенина,8,1,Санкт-Петербург Есенина 8 1,"(60.0386674, 30.3290738)"
2,Иван,Санкт-Петербург,Федора Абрамова,1,,Санкт-Петербург Федора Абрамова 1,"(60.0709429, 30.3357944)"
3,Пётр,Санкт-Петербург,Ленина,5,,Санкт-Петербург Ленина 5,"(59.96055, 30.309127)"


**Находим центральную точку**



---




Центральная точка (Барицентр) - это среднее арифметическое положений всех точек координат пользователей.

In [47]:
#Центральная точка (Барицентр)

center_point = center(list(adress_book['Coordinates']))
center_point

[60.02338676666667, 30.324665066666668]

**Ищем ближайший бар**



---



Находим ближайший бар к центральной точке

In [48]:
# Адрес поисковика бара

bar = requests.get('https://www.google.ru/maps/search/bar/@' + f'{center_point[0]}' + ','  + f'{center_point[1]}' + ',15z' )

In [49]:
#Название ближайшего бара (Если есть)
try:
  bar_name = re.findall(r'".+Owner', bar.text[40000:50000])[0][-50:]
  bar_name = re.findall(r',\\".+',bar_name)[0][3:-7]
except:
  pass

In [50]:
#Достаём адрес бара

bar_txt = re.findall(r',\\".+', bar.text[40000:50000])[0][3:]

adress_bar = re.findall(r'^[^"]*',bar_txt)

In [51]:
#Делаем запрос по адресу
r_bar = requests.get('https://www.google.com/maps/search/' + adress_bar[0])

#Регулярными выражениями находим координаты в тексте запроса
coord_bar = re.findall(r'center=.+?&', r_bar.text)[0][7:-1]

#Разделяем координаты
latitude_bar = re.findall(r'.+%', coord_bar)[0][:-1]
longitude_bar = re.findall(r'C.+', coord_bar)[0][1:]

In [52]:
#Координаты бара и название

print(f' Latitude: {latitude_bar}', '\n',
      f'Longitude: {longitude_bar}', '\n',
      f'Name of the bar: {bar_name}')

 Latitude: 60.0261108 
 Longitude: 30.3201399 
 Name of the bar: ROXETTE BAR


**Итоговая часть**



---



Наносим на карту все необходимые метки.

In [53]:
#Наносим координаты на карту

m = folium.Map(location=[center_point[0], center_point[1]], zoom_start=15)

#Цвет маркеров
icon_center = folium.Icon(color="green")
icon_bar = folium.Icon(color="red")

#Имя бара
if bar_name != None:
  b_name = bar_name
else:
  b_name = 'BAR'


#Маркер центра
folium.Marker((float(latitude_bar), float(longitude_bar)), tooltip=b_name, icon=icon_bar).add_to(m)


#Маркер бара
folium.Marker((center_point[0], center_point[1]), tooltip="Центр", icon=icon_center).add_to(m)

#Маркеры адресов
for i in range(len(adress_book)):
  folium.Marker((float(adress_book.loc[i+1, 'Coordinates'][0]), float(adress_book.loc[i+1, 'Coordinates'][1])), tooltip=adress_book.loc[i+1, 'User_Name']).add_to(m)

In [54]:
m



---



**Построение ближайшего маршрута** (В стадии разработки)


Построение пешего маршрута от центральной точки встречи до ближайшего бара

In [ ]:
ox.config(log_console=True, use_cache=True)

#Начальные и конечные координаты маршрута
start_point = (center_point[0], center_point[1])
end_point = (float(latitude_bar), float(longitude_bar))

#Параметры маршрута
place = 'Санкт-Петербург' #Необходимо вручную задать город центральной точки (в последствии исправлю)
mode = 'walk'
optimizer = 'time'

#Построение карты
graph = ox.graph_from_place(place, network_type = mode)

#Ближайшая точка к стартовой координате
start_nearest_point = ox.get_nearest_node(graph, start_point)
#Ближайшая точка к конечной координате
end_nearest_point = ox.get_nearest_node(graph, end_point)

#Короткий путь пешком
shortest_route = nx.shortest_path(graph, 
                                  start_nearest_point, 
                                  end_nearest_point, 
                                  weight=optimizer)

#Нанесение пути на карту
shortest_route_map = ox.plot_route_folium(graph, shortest_route)

warnings.simplefilter("ignore")

In [62]:
#Отобразить маршрут на карте

shortest_route_map